### Assignment : Week 1 
## Modeling simple RL problems by making their MDPs in Python

We will create the MDPs for some of the example problems from Grokking textbook. For the simple environments, we can just hardcode the MDPs into a dictionary by exhaustively encoding the whole state space and the transition function. We will also go through a more complicated example where the state space is too large to be manually coded and we need to implement the transition function based on some state parameters.

Later on, you will not need to implement the MDPs of common RL problems yourself, most of the work is already done by the OpenAI Gym library, which includes models for most of the famous RL envis.

You can start this assignment during/after reading Grokking Ch-2.

## Environment 0 - Bandit Walk

Let us consider the BW environment on Page 39. 

State Space has 3 elements, states 0, 1 and 2.
States 0 and 2 are terminal states and state 1 is the starting state.

Action space has 2 elements, left and right.

The environment is deterministic - transition probability of any action is 1.

Only 1 (State, Action, State') tuple has positive reward, (1, Right, 2) gives the agent +1 reward.

We'll model this MDP as a dictionary. This code is an example for the upcoming exercises.

In [12]:
bw_mdp = {

    0 : {
        "Right" : [(1, 0, 0, True)],
        "Left" : [(1, 0, 0, True)]
    },

    1 : {
        "Right" : [(1, 2, 1, True)],
        "Left" : [(1, 0, 0, True)]
    },

    2 : {
        "Right" : [(1, 2, 0, True)],
        "Left" : [(1, 2, 0, True)]
    }
    
}

Note that by convention, all actions from terminal states still lead to the same state with reward 0.

## Environment 1 - Slippery Walk

Let us now look at the BSW environment on Page 40. We'll model a slightly modified version of BSW with 7 states instead (i.e the SWF envi on Page 67). It will be useful in the coming weeks.

Here, states 0 and 6 are terminal states and state 3 is the starting state.

Action space has again 2 elements, left and right.

The environment is now stochastic, transition probability of any action is as follows -
If agent chooses `Right` at a non-terminal state,
- $50\%$ times it will go to `Right` state
- $33\frac{1}{3} \%$ times it will stay in same state
- $16\frac{2}{3}\%$ times it will go to `Left`state

This time, 2 different (State, Action, State') tuples have positive rewards, you need to find them.

We'll again model this MDP as a dictionary. Part of the code is written for you.

In [13]:
swf_mdp = {}


swf_mdp[0] = {
    "Right": [(1.0, 0, 0, True)],
    "Left":  [(1.0, 0, 0, True)]
}

swf_mdp[6] = {
    "Right": [(1.0, 6, 0, True)],
    "Left":  [(1.0, 6, 0, True)]
}

swf_mdp[1] = {
    "Right": [
        (1/2, 2, 0, False), 
        (1/3, 1, 0, False), 
        (1/6, 0, 0, True)   
    ],
    "Left": [
        (1/2, 0, 0, True),  
        (1/3, 1, 0, False), 
        (1/6, 2, 0, False)  
    ]
}


for s in range(2, 6):
    is_left_terminal = (s - 1 == 0)
    is_right_terminal = (s + 1 == 6)
    
   
    reward_val = 1.0 if (s + 1 == 6) else 0.0
    
    swf_mdp[s] = {
        "Right": [
            (1/2, s + 1, reward_val, is_right_terminal),
            (1/3, s,     0,          False),            
            (1/6, s - 1, 0,          is_left_terminal)  
        ],
        "Left": [
            (1/2, s - 1, 0,          is_left_terminal), 
            (1/3, s,     0,          False),            
            (1/6, s + 1, reward_val, is_right_terminal) 
        ]
    }


Feel free to automate filling this MDP, but ensure that it is correctly filled as it'll be back in next week's assignment.

## Environment 2 - Frozen Lake Environment

This environment is described on Page 46.

The FL environment has a large state space, so it's better to generate most of the MDP via Python instead of typing stuff manually.

Note that all 5 states - 5, 7, 11, 12, 15 are terminal states, so keep that in mind while constructing the MDP.

There are 4 actions now - Up, Down, Left, Right.

The environment is stochastic, and states at the border of lake will require separate treatment.



Yet again we will model this MDP as a (large) dictionary.

In [14]:
fl_mdp = {}


holes = [5, 7, 11, 12]
goal = 15
terminals = holes + [goal]


actions = {"Left": (0, -1), "Down": (1, 0), "Right": (0, 1), "Up": (-1, 0)}


slips = {
    "Left":  ["Up", "Down"],
    "Down":  ["Left", "Right"],
    "Right": ["Up", "Down"],
    "Up":    ["Left", "Right"]
}

for s in range(16):
    fl_mdp[s] = {}
    
    
    if s in terminals:
        for a in actions:
            fl_mdp[s][a] = [(1.0, s, 0, True)]
        continue

    
    row, col = s // 4, s % 4
    for a, (dr, dc) in actions.items():
        outcomes = []
       
        possible_moves = [a] + slips[a]
        
        for move in possible_moves:
            mdr, mdc = actions[move]
            nr, nc = row + mdr, col + mdc
            
            
            if 0 <= nr < 4 and 0 <= nc < 4:
                next_s = nr * 4 + nc
            else:
                next_s = s
            
            is_done = next_s in terminals
            reward = 1.0 if next_s == goal else 0.0
            outcomes.append((1/3, next_s, reward, is_done))
            
        fl_mdp[s][a] = outcomes

In [15]:
for state in range(0, 16):
    
    transitions = {}
    
   
    holes = [5, 7, 11, 12]
    goal = 15
    
    
    if state in holes or state == goal:
        for action in ["Up", "Down", "Right", "Left"]:
            transitions[action] = [(1.0, state, 0, True)]
        fl_mdp[state] = transitions
        continue

   
    row, col = state // 4, state % 4

    for action in ["Up", "Down", "Right", "Left"]:
        
        if action == "Up" or action == "Down":
            possible_actions = [action, "Left", "Right"]
        else:
            possible_actions = [action, "Up", "Down"]
            
        outcomes = []
        for act in possible_actions:
            
            new_row, new_col = row, col
            if act == "Up":    new_row = max(0, row - 1)
            if act == "Down":  new_row = min(3, row + 1)
            if act == "Left":  new_col = max(0, col - 1)
            if act == "Right": new_col = min(3, col + 1)
            
            new_state = new_row * 4 + new_col
            
            
            done = (new_state in holes or new_state == goal)
            reward = 1.0 if new_state == goal else 0.0
            
            outcomes.append((1/3, new_state, reward, done))
            
        transitions[action] = outcomes

    fl_mdp[state] = transitions

You might need to do some stuff manually, but make sure to automate most of it.

You can check your implementation of the FL environment by comparing it with the one in OpenAI Gym.

You don't need to worry about Gym right now, we'll set it up in the coming weeks. But here is the code to import an MDP.

In [24]:
#!pip install gymnasium
#import gymnasium as gym
#P = gym.make('FrozenLake-v1').env.P

import gymnasium as gym
import pprint


env = gym.make('FrozenLake-v1')
P = env.unwrapped.P


Since the imported MDP is also just a dictionary, we can just print it.

In [23]:
# using the pretty print module

print("Success")
pprint.pprint(P[14])

Successfully accessed the MDP! Transitions for State 14:
{0: [(0.33333333333333337, 10, 0, False),
     (0.3333333333333333, 13, 0, False),
     (0.33333333333333337, 14, 0, False)],
 1: [(0.33333333333333337, 13, 0, False),
     (0.3333333333333333, 14, 0, False),
     (0.33333333333333337, 15, 1, True)],
 2: [(0.33333333333333337, 14, 0, False),
     (0.3333333333333333, 15, 1, True),
     (0.33333333333333337, 10, 0, False)],
 3: [(0.33333333333333337, 15, 1, True),
     (0.3333333333333333, 10, 0, False),
     (0.33333333333333337, 13, 0, False)]}
